# Connecting to the database

And selecting the unique elements of the products

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('inventory.db')
cursor = conn.cursor()

# Fetch unique item names
cursor.execute("SELECT DISTINCT item_name FROM products")
items = [row[0] for row in cursor.fetchall()]

# Importing genai

In [ ]:
import google.generativeai as genai
genai.configure(api_key='YOUR_API_KEY')

# To retrieve user requests from prompt<br>and create SQL query

In [106]:
model = genai.GenerativeModel('models/gemini-1.5-flash')

# User query
user_query = "How many shirts are availble on the store ?"

# Prompt the model to generate a SQL WHERE clause
prompt = f"""
Given the user query: "{user_query}", generate a SQL clause to filter the products table for items and select what the user is requesting.
here's the available columns item_name ,quantity ,price ,color 

Here's the ALL THE items available on the store : {items}

Respond only with the SQL code if the item is available on the store for example here's a code : SELECT DISTINCT item_name FROM products.
If the request of the user isn't on the available items on the store from the given items you must return No_result_found
"""

response = model.generate_content(prompt)

In [107]:
response.text

"```sql\nSELECT SUM(quantity) AS total_shirts \nFROM products\nWHERE item_name = 'Shirt' OR item_name = 'Polo Shirt';\n```\n"

# Cleaning the return

In [108]:
def extract_sql_query(text):
    # Remove leading and trailing whitespace
    text = text.strip()
    
    # Remove code block backticks if present
    if text.startswith('```sql') and text.endswith('```'):
        text = text.strip('```').replace('sql\n', '').strip()
    
    return text

# Example usage

sql_query = extract_sql_query(response.text)
print(sql_query)

SELECT SUM(quantity) AS total_shirts 
FROM products
WHERE item_name = 'Shirt' OR item_name = 'Polo Shirt';


# Filtering the database

In [109]:
conn = sqlite3.connect('inventory.db')
cursor = conn.cursor()

cursor.execute(sql_query)
result = [row[0] for row in cursor.fetchall()]

In [110]:
result

[195]

# Final request to the LLM with filtered data

In [ ]:
# Prompt the model to generate a SQL WHERE clause
prompt = f"""
You're a shop assitant , please answer the user based on the retrieved 
documents they have been retrieved from the database and it's the informations about 
what the user requested.
Here's the original user query {user_query}
and here's the retrieved documents from the database {result}
if the result is not empty it's a confirmation that the request of the user is available
"""

response = model.generate_content(prompt)

In [112]:
response.text

'We currently have 195 shirts available in the store.\n'